Comienzo importando los módulos .py que armé:

In [1]:
import sys
import os
import pprint

# Implemento esta línea para generar acceso a los módulos
sys.path.append(os.path.join(os.getcwd(), "src"))

from evaluar import calcular_puntaje, mejor_equipo_ronda
from acumular import ranking_innovacion, inicializar_acumulados, actualizar_acumulados, marcar_mejor_equipo
from tabla import tabla_ronda, tabla_resultados_finales, tabla_ranking_innovacion


La estructura principal con la que trabajo es esta lista de diccionarios llamada "Evaluaciones"

In [2]:
evaluaciones = [
 # Ronda 1
 {
 'EquipoA': {'innovacion': 2, 'presentacion': 1, 'errores': True},
 'EquipoB': {'innovacion': 1, 'presentacion': 0, 'errores': False},
 'EquipoC': {'innovacion': 1, 'presentacion': 2, 'errores': True},
 'EquipoD': {'innovacion': 0, 'presentacion': 1, 'errores': False},
 'EquipoE': {'innovacion': 1, 'presentacion': 1, 'errores': False}
 },
 # Ronda 2
 {
 'EquipoA': {'innovacion': 0, 'presentacion': 2, 'errores': False},
 'EquipoB': {'innovacion': 2, 'presentacion': 0, 'errores': True},
 'EquipoC': {'innovacion': 1, 'presentacion': 1, 'errores': False},
 'EquipoD': {'innovacion': 2, 'presentacion': 1, 'errores': True},
 'EquipoE': {'innovacion': 0, 'presentacion': 1, 'errores': False}
 },
 # Ronda 3
 {
 'EquipoA': {'innovacion': 3, 'presentacion': 2, 'errores': False},
 'EquipoB': {'innovacion': 1, 'presentacion': 1, 'errores': True},
 'EquipoC': {'innovacion': 2, 'presentacion': 0, 'errores': False},
 'EquipoD': {'innovacion': 1, 'presentacion': 3, 'errores': True},
 'EquipoE': {'innovacion': 2, 'presentacion': 2, 'errores': False}
 },
 # Ronda 4
 {
 'EquipoA': {'innovacion': 1, 'presentacion': 3, 'errores': True},
 'EquipoB': {'innovacion': 2, 'presentacion': 2, 'errores': False},
 'EquipoC': {'innovacion': 3, 'presentacion': 1, 'errores': False},
 'EquipoD': {'innovacion': 0, 'presentacion': 2, 'errores': True},
 'EquipoE': {'innovacion': 2, 'presentacion': 0, 'errores': False}
 },
 # Ronda 5
 {
 'EquipoA': {'innovacion': 2, 'presentacion': 2, 'errores': False},
 'EquipoB': {'innovacion': 1, 'presentacion': 3, 'errores': True},
 'EquipoC': {'innovacion': 0, 'presentacion': 2, 'errores': False},
 'EquipoD': {'innovacion': 3, 'presentacion': 1, 'errores': False},
 'EquipoE': {'innovacion': 2, 'presentacion': 3, 'errores': True}
 }
]

A continuación, implemento las funciones para resolver la actividad. 

Para empezar, creo el diccionario de "acumulados", para llevar el acumulado de todas las rondas.

In [3]:
# Creo el diccionario de acumulados
acumulados = inicializar_acumulados(evaluaciones) 

Recorro todas las rondas con sus equipos para calcular y luego informar en una tabla:
- Puntaje de cada equipo en cada ronda
- Mejor equipo de cada ronda
- Ordenar para cada ronda, los equipos de forma descendente según su puntaje

A su vez, también desarrollo para luego informar:
- Actualizar "acumulados"
- Determinar ronda con mayor puntaje

In [4]:
# Recorrido de toda la estructura "evaluaciones", recorriendo todas sus rondas
for i, ronda in enumerate(evaluaciones, start=1):       # Enumerate es función incrporada que agrega contador a iterable (para imprimir ronda).REEMPLAZA variable contadora i
    dict_ronda = ronda.copy()                           # Lógica: genero una copia de la ronda para no modificar "evaluaciones"
    mejor_puntaje = -1         
    mejor_nombre = None        
    mayor_total_puntaje = -1
    nro_ronda_mayor_puntaje = None
    
    # Recorrido de cada ronda, recorriendo todos sus equipos
    for nombre_equipo, datos_equipo in ronda.items():
        dict_equipo = datos_equipo.copy()                       # Lógica: genero copia del equipo para no modificar original
        dict_equipo["puntaje"] = calcular_puntaje(dict_equipo)  # Agrego puntaje en la copia

        # Busco el resultado actualizado de mejor equipo
        mejor_nombre, mejor_puntaje = mejor_equipo_ronda(nombre_equipo, dict_equipo, mejor_puntaje, mejor_nombre)
        
        # Agrego copia equipo a copia de ronda
        dict_ronda[nombre_equipo] = dict_equipo     

        # Por ahora actualizo acumulados sin contador de si fueron "mejores"
        actualizar_acumulados(acumulados, dict_equipo, nombre_equipo) 
    

    #Ordeno equipo por puntaje descendente
    equipos_ordenados = sorted(
        dict_ronda.items(),
        key=lambda item: item[1]['puntaje'],                #lambda, FUNCIONES ANÓNIMAS (video)
        reverse=True
    )

    dict_ronda = {
    nombre: {**datos, 'mejores': nombre == mejor_nombre} for nombre, datos in equipos_ordenados     # Dict comprehension agregandole el dato "mejor", con KWARGS (explicar en video),"podría haber hecho una función y usar map peroes mejor"
    }    

    
    # Agrego a "acumulados" info. contadora de "mejores" por fuera del bucle de equipos, para ahorrar un for
    marcar_mejor_equipo(acumulados, mejor_nombre)

    # Sumar los puntajes de todos los equipos de esta ronda, actividad extra 3
    total_puntaje_ronda = sum(equipo["puntaje"] for equipo in dict_ronda.values())
    if total_puntaje_ronda > mayor_total_puntaje:
        mayor_total_puntaje = total_puntaje_ronda
        nro_ronda_mayor_puntaje = i

    # Imprimo la tabla de la ronda
    tabla_ronda(i, dict_ronda)
    
    # TERMINA RECORRIDO EQUIPOS DE UNA RONDA

# TERMINA RECORRIDO DE TODAS LAS RONDAS


 Ronda 1 : 
--------------------------------------------------
Equipo     Innov  Pres   Errores  Puntaje  Mejor 
--------------------------------------------------
EquipoA    2      1      True     6        True  
EquipoC    1      2      True     4        False 
EquipoE    1      1      False    4        False 
EquipoB    1      0      False    3        False 
EquipoD    0      1      False    1        False 

 Ronda 2 : 
--------------------------------------------------
Equipo     Innov  Pres   Errores  Puntaje  Mejor 
--------------------------------------------------
EquipoD    2      1      True     6        True  
EquipoB    2      0      True     5        False 
EquipoC    1      1      False    4        False 
EquipoA    0      2      False    2        False 
EquipoE    0      1      False    1        False 

 Ronda 3 : 
--------------------------------------------------
Equipo     Innov  Pres   Errores  Puntaje  Mejor 
--------------------------------------------------
Equip

Luego de recorrer todas las rondas:
- Armo ranking de puntos de innovación
- Informo ranking, tabla de acumulados (resaltando al ganador) y ronda con mayor puntaje

In [5]:
# Teniendo información de todas las rondas, armo el ranking de puntos de innovación
ranking = ranking_innovacion(acumulados)

# Ordenar los acumulados por puntaje descendente
acumulados_ordenados = sorted(
    acumulados.items(),
    key=lambda item: item[1]['puntaje'],
    reverse=True
)

# Imprimo la tabla de los resultados finales:
tabla_resultados_finales(acumulados_ordenados)

# Imprimo ranking de innovación: 
tabla_ranking_innovacion(ranking)
        
# Imprimo ronda con mayor puntaje (no creo una función del módulo "tabla" porque es poca información)
print(f"\n Ronda con mayor puntaje total: Ronda {nro_ronda_mayor_puntaje} con {mayor_total_puntaje} puntos.")



 Resultados Finales
--------------------------------------------------
Equipo     Innov    Pres     Errores  Puntaje  Mejores 
--------------------------------------------------
EquipoA    8        10       2        30       2              
EquipoE    7        7        1        26       0              
EquipoC    7        6        1        25       1              
EquipoB    7        6        3        21       0              
EquipoD    6        8        3        20       2              
El ganador fue el EquipoA

 Ranking Puntos Innovacion Acumulados
--------------------------------------------------
Equipo     Puntos Innovacion
--------------------------------------------------
EquipoA    24      
EquipoB    21      
EquipoC    21      
EquipoE    21      
EquipoD    18      

 Ronda con mayor puntaje total: Ronda 5 con 33 puntos.
